In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import random
import os
import numpy as np
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
import torch
test_path = "C:/Users/22668/Desktop/github/sin-hoyeon/open/test.csv"
train_path = "C:/Users/22668/Desktop/github/sin-hoyeon/open/train.csv"

In [2]:
train = pd.read_csv(train_path).drop(columns = ["id"])
train_len = len(train)
test = pd.read_csv(test_path)
id_test = test["id"]
test = pd.read_csv(test_path).drop(columns = ["id"])

In [3]:
dataset = pd.concat([train,test],axis=0)
dataset = dataset.drop(columns = ["father","mother","gender"])

In [4]:
dataset

,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15,class
0,2,G G,A G,A A,G A,C A,A A,A A,G G,A A,G G,A G,A A,A A,A A,A A,B
1,2,A G,A G,C A,A A,A A,A G,A A,G A,A A,A G,A A,G A,G G,A A,A A,C
2,2,G G,G G,A A,G A,C C,G G,A A,G A,G A,A G,A A,A A,A A,A A,A A,B
3,1,A A,G G,A A,G A,A A,G G,G G,A A,G G,A G,G G,G G,G G,A A,G G,A
4,2,G G,G G,C C,A A,C C,A A,A A,A A,A A,G G,A A,A A,A G,A A,G A,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,2,A G,G G,C C,A A,C A,A G,A A,G G,A A,G G,G G,A A,A A,A A,G A,NaN
171,2,G G,A A,A A,A A,C A,A G,A A,A A,A A,A G,A A,A A,A G,A A,G A,NaN
172,2,G G,A A,A A,A A,C A,A G,A A,A A,A A,G G,A G,A A,A G,A A,G G,NaN
173,2,A G,G G,C A,G A,C C,G G,A A,G A,A A,G G,A G,A A,A A,A A,A A,NaN


In [5]:
_t = []
for val in dataset.SNP_01 == "G G":
    if val == True:
        _t.append(1)
    else:
        _t.append(0)
dataset["has01GG"] = _t

_t = []
for val in dataset.SNP_02 == "A A":
    if val == True:
        _t.append(1)
    else:
        _t.append(0)
dataset["has02AA"] = _t

In [6]:
def create_col(dataset,col,value):
    _t = []
    for val in dataset[col] == value:
        if val == True:
            _t.append(1)
        else:
            _t.append(0)
    
    col_name_base = "has"+col[-2:]
    value_name = ""
    for chr in value:
        if chr != " ":
            value_name+=chr
    col_name = col_name_base+value_name
    print(col_name)
    dataset[col_name] = _t

    return dataset

dataset = create_col(dataset,"SNP_03","A A")
dataset = create_col(dataset,"SNP_04","G G")
dataset = create_col(dataset,"SNP_05","C C")
dataset = create_col(dataset,"SNP_06","A A")
dataset = create_col(dataset,"SNP_07","A A")
dataset = create_col(dataset,"SNP_07","G G")
dataset = create_col(dataset,"SNP_08","G G")

dataset = create_col(dataset,"SNP_09","A A")
dataset = create_col(dataset,"SNP_09","G G")
dataset = create_col(dataset,"SNP_11","A A")

dataset = create_col(dataset,"SNP_12","A A")
dataset = create_col(dataset,"SNP_12","G G")

dataset = create_col(dataset,"SNP_13","A A")
dataset = create_col(dataset,"SNP_14","A A")

has03AA
has04GG
has05CC
has06AA
has07AA
has07GG
has08GG
has09AA
has09GG
has11AA
has12AA
has12GG
has13AA
has14AA


In [7]:
dataset = dataset.drop(columns = "SNP_03")

In [8]:
#one-hot encoding for distance base algorithm
dataset_ohe = pd.get_dummies(dataset,columns = dataset.columns.drop("class"),drop_first=True) #multicollinearity를 막기위한 drop_first 옵션
train_ohe = dataset_ohe[:train_len].copy()
test_ohe = dataset_ohe[train_len:].copy().drop(columns="class")

class_map = {"A":0,"B":1,"C":2}
train_ohe["class"]=train_ohe["class"].map(class_map).astype(int)
X_train_ohe = train_ohe.drop(columns = "class")
Y_train_ohe = train_ohe["class"]

In [9]:
Y_train_ohe

0      1
1      2
2      1
3      0
4      2
      ..
257    1
258    2
259    0
260    0
261    1
Name: class, Length: 262, dtype: int32

In [10]:
import torch
import torch.nn as nn


In [11]:
X_train_ohe = torch.from_numpy(X_train_ohe.values).float()
#y_train_ohe = torch.from_numpy(pd.get_dummies(Y_train_ohe).values).float()
Y_train_ohe = torch.from_numpy(Y_train_ohe.values).long()

In [12]:
class mynet(nn.Module):
    def __init__(self,in_features,dropout_p,l1_out=64):
        super().__init__()
        self.linr1 = torch.nn.Linear(in_features,l1_out)
        self.relu1 = torch.nn.ReLU()
        self.d1 = torch.nn.Dropout(p=dropout_p)
        self.b1 = torch.nn.BatchNorm1d(l1_out)
        # 256
        
        self.linr2 = torch.nn.Linear(l1_out,l1_out//2)
        self.relu2 = torch.nn.ReLU()
        self.d2 = torch.nn.Dropout(p=dropout_p)
        self.b2 = torch.nn.BatchNorm1d(l1_out//2)
        # 128

        self.linr3 = torch.nn.Linear(l1_out//2,l1_out//4)
        self.relu3 = torch.nn.ReLU()
        self.d3 = torch.nn.Dropout(p=dropout_p)
        self.b3 = torch.nn.BatchNorm1d(l1_out//4)
        # 64
        self.linr4 = torch.nn.Linear(l1_out//4,l1_out//8)
        self.relu4 = torch.nn.ReLU()
        self.d4 = torch.nn.Dropout(p=dropout_p)
        self.b4 = torch.nn.BatchNorm1d(l1_out//8)
        # 32
        self.linr5 = torch.nn.Linear(l1_out//8,l1_out//16)
        self.relu5 = torch.nn.ReLU()
        self.d5 = torch.nn.Dropout(p=dropout_p)
        self.b5 = torch.nn.BatchNorm1d(l1_out//16)
        #16
        self.linr6 = torch.nn.Linear(l1_out//16,3)

    def forward(self,x):
        out = self.b1(self.d1(self.relu1(self.linr1(x))))
        out = self.b2(self.d2(self.relu2(self.linr2(out))))
        out = self.b3(self.d3(self.relu3(self.linr3(out))))
        out = self.b4(self.d4(self.relu4(self.linr4(out))))
        out = self.b5(self.d5(self.relu5(self.linr5(out))))
        out = self.linr6(out)
        #out = self.b3(self.d3(self.relu3(self.linr3(x))))
        return out

In [50]:
from sklearn.model_selection import StratifiedKFold
import random
epochs_list = [i for i in range(1400,2000,20)]
weight_decay = np.linspace(0.001,0.0001,500).tolist()
lr = np.linspace(1e-2,1e-5,500).tolist()
hidden_nodes = [i for i in range(1300,1800)]
dropout_p = np.linspace(0.6,0.8,1000).tolist()
rs = [i for i in range(0,500)]

In [51]:
def dl_cv(epochs,weight_decay,learning_rate,hidden1_nodes,dropout_p,rs):

    try_number=0
    skf = StratifiedKFold(n_splits=5,shuffle=True)
    skf.get_n_splits(X_train_ohe,Y_train_ohe)
    
    while True:
        try_number+=1
        print(f'try:{try_number}...')
        train_accs = []
        val_accs = []
        hdly1 = random.sample(hidden1_nodes,1)[0]
        lr = random.sample(learning_rate,1)[0]
        wght_decay = random.sample(weight_decay,1)[0]
        epoch = random.sample(epochs,1)[0]
        drop_p = random.sample(dropout_p,1)[0]
        r_seed = random.sample(rs,1)[0]
        print(f'lr:{lr} wght_decay:{wght_decay} epochs:{epoch} hidden_l1nodes:{hdly1} dropout_prob:{drop_p}')
        for train_index,valid_index in skf.split(X_train_ohe,Y_train_ohe): 
            torch.manual_seed(r_seed)
            net = mynet(45,drop_p,l1_out=hdly1)
            optimizer = torch.optim.Adam(net.parameters(),lr=lr,weight_decay = wght_decay)        
            loss_fn = torch.nn.CrossEntropyLoss()
            #KFold
            train_index = train_index.tolist();valid_index = valid_index.tolist()
            X_tr = X_train_ohe[train_index,:];y_tr = Y_train_ohe[train_index]
            X_tst = X_train_ohe[valid_index,:];y_valid = Y_train_ohe[valid_index]

            #fitting
            for ep in range(epoch):
                net.train()
                #1 yhat
                yhat = net(X_tr)
                #2 loss
                loss = loss_fn(yhat,y_tr)
                if ep % 50 == 0:
                    pass
                    #print(loss)
                #3 derivative
                loss.backward()
                #4 update
                optimizer.step()
                optimizer.zero_grad()
            train_yhat = torch.argmax(net(X_tr),dim=1)
            train_acc = torch.mean((train_yhat==y_tr).float())
            print("trainacc : ",train_acc)
            train_accs.append(train_acc)
            net.eval()
            with torch.no_grad():
                val_yhat = torch.argmax(net(X_tst),dim=1)
                val_acc = torch.mean((val_yhat == y_valid).float()).tolist()
                print("validacc : ",val_acc)
                val_accs.append(val_acc)
            if val_acc < 0.97:
                break

        valid_accuracy = torch.mean(torch.tensor(val_accs))
        print(f"K-Fold train accuracy {torch.mean(torch.tensor(train_accs))}")
        print(f"K-Fold valid accuracy {torch.mean(torch.tensor(val_accs))}")
        print("==========================================================")
        if valid_accuracy > 0.99:
            path = "./model{}.pth".format(try_number)
            torch.save(net,path)


In [52]:
t = dl_cv(epochs_list,weight_decay,lr,hidden_nodes,dropout_p,rs)

try:1...
lr:0.007377374749498998 wght_decay:0.0003597194388777555 epochs:1480 hidden_l1nodes:1609 dropout_prob:0.6712712712712713
trainacc :  tensor(0.9952)
validacc :  0.9433962106704712
K-Fold train accuracy 0.9952152967453003
K-Fold valid accuracy 0.9433962106704712
try:2...
lr:0.0031931863727454905 wght_decay:0.00048597194388777553 epochs:1900 hidden_l1nodes:1639 dropout_prob:0.786986986986987
trainacc :  tensor(0.9952)
validacc :  0.9433962106704712
K-Fold train accuracy 0.9952152967453003
K-Fold valid accuracy 0.9433962106704712
try:3...
lr:0.00439438877755511 wght_decay:0.0006266533066132265 epochs:1920 hidden_l1nodes:1583 dropout_prob:0.7737737737737738
trainacc :  tensor(0.9952)
validacc :  0.9245283007621765
K-Fold train accuracy 0.9952152967453003
K-Fold valid accuracy 0.9245283007621765
try:4...
lr:0.0018118036072144288 wght_decay:0.00023166332665330663 epochs:1960 hidden_l1nodes:1427 dropout_prob:0.6944944944944945
trainacc :  tensor(1.)
validacc :  0.9056603908538818
K-Fo

KeyboardInterrupt: 

# Assemble

In [66]:
test_ohe = torch.from_numpy(test_ohe.values).float()

In [52]:
import os
path = "C:/Users/22668/Desktop/새 폴더"
models = []
for model in os.listdir(path):
    md_path = os.path.join(path,model)
    models.append(torch.load(md_path))

In [93]:
assemble = torch.zeros(test_ohe.shape[0],3)
assemble.shape

torch.Size([175, 3])

In [94]:
soft = torch.nn.Softmax(dim=1)

In [95]:
for model in models:
    net = model
    yhat = soft(model(test_ohe))
    assemble +=yhat

In [102]:
test_predict = torch.argmax(assemble,dim=1)
result = pd.concat([pd.Series(id_test),pd.Series(test_predict)],axis=1)
result.columns = ["id","class"]
class_map_inv = {0:"A",1:"B",2:"C"}
result["class"] = result["class"].map(class_map_inv)

In [103]:
result

,id,class
0,TEST_000,A
1,TEST_001,B
2,TEST_002,C
3,TEST_003,B
4,TEST_004,A
...,...,...
170,TEST_170,B
171,TEST_171,C
172,TEST_172,C
173,TEST_173,B


In [104]:
result.to_csv("./submission_dl.csv",index=False)